In [1]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# Dataset Paths
base_dir = r'D:\Petponks\data'
train_dir = r'D:\Petponks\data_split\train'
val_dir = r'D:\Petponks\data_split\val'

In [4]:
# Function to split data
def split_data(source_dir, train_dir, val_dir, split_ratio=0.2):
    for class_name in os.listdir(source_dir):
        class_dir = os.path.join(source_dir, class_name)
        if os.path.isdir(class_dir):
            images = [img for img in os.listdir(class_dir) if img.endswith(('.jpg', '.jpeg', '.png'))]
            train_images, val_images = train_test_split(images, test_size=split_ratio, random_state=42)
            
            # Create directories
            os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
            os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
            
            # Copy images
            for img in train_images:
                shutil.copy(os.path.join(class_dir, img), os.path.join(train_dir, class_name, img))
            for img in val_images:
                shutil.copy(os.path.join(class_dir, img), os.path.join(val_dir, class_name, img))

    print(f"Data split complete. Training images: {sum(len(os.listdir(os.path.join(train_dir, d))) for d in os.listdir(train_dir))}")
    print(f"Validation images: {sum(len(os.listdir(os.path.join(val_dir, d))) for d in os.listdir(val_dir))}")

In [5]:
# Split the data
split_data(base_dir, train_dir, val_dir)

Data split complete. Training images: 350
Validation images: 89


In [6]:
# Image Parameters
img_size = 299  # InceptionV3 default input size
batch_size = 32

In [7]:
# Image Data Generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [8]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
# Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 350 images belonging to 4 classes.


In [10]:

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 89 images belonging to 4 classes.


In [11]:
# Print class indices
print("Class indices:", train_generator.class_indices)

Class indices: {'Bacterial_dermatosis': 0, 'Fungal_infections': 1, 'Healthy': 2, 'Hypersensitivity_allergic_dermatosis': 3}


In [12]:
# Load the pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

In [13]:
# Freeze the base model layers
base_model.trainable = False

In [14]:
# Model Definition
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

In [15]:
# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [16]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)


In [17]:
# Training the Model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=100,  # Increase epochs, early stopping will prevent overfitting
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/100
11/11 [==============================] - 23s 1s/step - loss: 2.0479 - accuracy: 0.3400 - val_loss: 2.0660 - val_accuracy: 0.2809 - lr: 0.0010
Epoch 2/100
11/11 [==============================] - 8s 719ms/step - loss: 1.6693 - accuracy: 0.4343 - val_loss: 1.5478 - val_accuracy: 0.4382 - lr: 0.0010
Epoch 3/100
11/11 [==============================] - 8s 724ms/step - loss: 1.3793 - accuracy: 0.5371 - val_loss: 1.6649 - val_accuracy: 0.4157 - lr: 0.0010
Epoch 4/100
11/11 [==============================] - 8s 701ms/step - loss: 1.3363 - accuracy: 0.5600 - val_loss: 1.7212 - val_accuracy: 0.4157 - lr: 0.0010
Epoch 5/100
11/11 [==============================] - 8s 725ms/step - loss: 1.1516 - accuracy: 0.5657 - val_loss: 1.2975 - val_accuracy: 0.4607 - lr: 0.0010
Epoch 6/100
11/11 [==============================] - 8s 702ms/step - loss: 1.0275 - accuracy: 0.6429 - val_loss: 1.2886 - val_accuracy: 0.5393 - lr: 0.0010
Epoch 7/100
11/11 [==============================] - 8s 684ms/step

In [18]:
# Model Evaluation
print("Evaluating model...")
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Evaluating model...
3/3 [==============================] - 1s 220ms/step - loss: 0.9554 - accuracy: 0.6292
Validation Accuracy: 62.92%


In [19]:
# Fine-tuning
base_model.trainable = True
for layer in base_model.layers[:249]:
    layer.trainable = False

In [20]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization_94 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               1

In [21]:
# Fine-tuning training
history_fine = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/50
11/11 [==============================] - 14s 837ms/step - loss: 0.9127 - accuracy: 0.7229 - val_loss: 1.0029 - val_accuracy: 0.6742 - lr: 1.0000e-04
Epoch 2/50
11/11 [==============================] - 8s 720ms/step - loss: 0.6731 - accuracy: 0.7800 - val_loss: 1.2064 - val_accuracy: 0.6517 - lr: 1.0000e-04
Epoch 3/50
11/11 [==============================] - 8s 701ms/step - loss: 0.5881 - accuracy: 0.7800 - val_loss: 1.1046 - val_accuracy: 0.6404 - lr: 1.0000e-04
Epoch 4/50
11/11 [==============================] - 8s 700ms/step - loss: 0.4688 - accuracy: 0.8314 - val_loss: 0.9646 - val_accuracy: 0.6966 - lr: 1.0000e-04
Epoch 5/50
11/11 [==============================] - 8s 688ms/step - loss: 0.4974 - accuracy: 0.8371 - val_loss: 0.9961 - val_accuracy: 0.7079 - lr: 1.0000e-04
Epoch 6/50
11/11 [==============================] - 8s 706ms/step - loss: 0.3251 - accuracy: 0.8914 - val_loss: 1.0821 - val_accuracy: 0.7079 - lr: 1.0000e-04
Epoch 7/50
11/11 [===========================

In [22]:
# Final Evaluation
print("Evaluating fine-tuned model...")
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Final Validation Accuracy: {val_accuracy * 100:.2f}%")

Evaluating fine-tuned model...
3/3 [==============================] - 1s 184ms/step - loss: 0.8171 - accuracy: 0.7640
Final Validation Accuracy: 76.40%


In [23]:
# Save the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization_94 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               1

In [24]:
# Save the model
model.save('improved_dog_disease_prediction_model.h5')